In [9]:
#quantipy for RIM and pandas for processing
import pandas as pd
import numpy as np
import quantipy as qp
import os

ModuleNotFoundError: No module named 'quantipy'

In [29]:
#parameters Initialization
Population = "69024"
Multiplier = "5"
ScaleDownFactor = "1"
Male = "48.4"
Female = "51.6"
Legacy = "True"

In [30]:
#Path to store the weighted data
path = "{}/processed/weighted/{}/{}/weighted_respondent.csv/".format(generic_path,Year,Quarter)
#Path to store backup files
backup_path = "{}/processed/weighted/{}/{}/weighted_respondent_backup".format(generic_path,Year,Quarter)
#Previously backuped files path
#c_backup_path = "{}/processed/weighted/{}/{}/weighted_respondent_backup/weighted_respondent.csv".format(generic_path,Year,Quarter)
mssparkutils.fs.mkdirs(backup_path)

In [31]:
#When reiterrating the process coppy files that are previously successful and create a backup  
try:    
    existing_files = mssparkutils.fs.ls(path)   
except:
    existing_files = None
    
#Delete the previous version of backup
files = mssparkutils.fs.ls(backup_path)
for file in files:
    if(file.isDir == False):
        mssparkutils.fs.rm("{}/{}".format(backup_path, file.name))
        #print(file.name)

In [32]:
#Backup previously executed files
if existing_files is not None:
    #mssparkutils.fs.cp(path, backup_path,True)
    for file in existing_files:
        mssparkutils.fs.cp(file.path, backup_path)
        #print(file.path)

In [33]:
###########################
# var definitions
###########################

POPULATION = int(Population)
QUARTER = "{}q{}".format(Year,Quarter)

# capping
multiplier = int(Multiplier)
scale_down_factor = int(ScaleDownFactor)
male_count=float(Male)
female_count=float(Female)

In [34]:
#check installed components(python - packages/libraries)
#import pkg_resources
#for d in pkg_resources.working_set:
#     print(d)

In [35]:
#Initializing the Source paths
#only the postalcode is parameter not initialized

#Card Data
#Card_Source_Path = "G:/Arimac Lanka/2018Q1/Card2018Q1.csv"
card_source_path = card_source_path = "{}/curated/card/{}/{}/card.csv".format(generic_path,Year,Quarter) # /part-00007-1a6bae43-ae54-4870-be63-205f66b430b5-c000.csv"

#respondant Data
#Respondant_SourcePath = "G:/Arimac Lanka/2018Q1/respondant.csv"
respondant_source_path = "{}/curated/respondent/{}/{}/respondent.csv".format(generic_path,Year,Quarter) #/part-00002-878212a3-89e7-4724-b8ff-e762fbcfe5d4-c000.csv"

#CardKind
#cardkind_source_path = "G:/Arimac Lanka/2018Q1/cardkind.csv"
cardkind_source_path = "{}/curated/cardkind/{}/{}/cardkind.csv".format(generic_path,Year, Quarter)

#Postal Codes
#PostalCodes_SourcePath = "G:/Arimac Lanka/2018Q1/postal_codes.csv"

# Value kept hardcoded as only Postal Codes are generated for 2018/04 quarter
postalCodes_source_path = "{}/curated/vpp-mysqldb/postal-service/regions/{}/{}/postal_codes.csv".format(generic_path,Year, Quarter)

# legacy respondent for missing data
if Legacy == "True":
    legacy_repondent_source_path = "{}/raw/legacy-pun/survey-response/respondent/{}/{}/respondent.csv".format(generic_path,Year,Quarter)


In [36]:
#Initializing targets source files

#RegionTarget
#Region Targets - Raw/2018/4/regiontargets.csv
#region_codes = "G:/Arimac Lanka/2018Q1/region-targets.csv"
region_codes_source_path = "{}/curated/vpp-mysqldb/weight-service/region/{}/4/region-targets.csv".format(generic_path,Year,Quarter)

#source = "G:/Arimac Lanka/2018Q1/household_comp-targets.csv"
#change agr to age after publishing and running the weights pipline
age_targets_source = "{}/curated/vpp-mysqldb/weight-service/age/{}/4/age-targets.csv".format(generic_path,Year)
household_comp_targets_source = "{}/curated/vpp-mysqldb/weight-service/household_comp/{}/4/household_comp-targets.csv".format(generic_path,Year)
income_targets_source = "{}/curated/vpp-mysqldb/weight-service/income/{}/4/income-targets.csv".format(generic_path,Year)
region_targets_source = "{}/curated/vpp-mysqldb/weight-service/region/{}/4/region-targets.csv".format(generic_path,Year)
race_targets_source = "{}/curated/vpp-mysqldb/weight-service/race/{}/4/race-targets.csv".format(generic_path,Year,Quarter)
amex_targets_source = "{}/curated/vpp-mysqldb/weight-service/amexownership/{}/4/amex_ownership-targets.csv".format(generic_path,Year)
visa_targets_source = "{}/curated/vpp-mysqldb/weight-service/visa_Ownership/{}/4/visa_ownership-targets.csv".format(generic_path,Year)
mastercard_targets_source = "{}/curated/vpp-mysqldb/weight-service/mastercard_ownership/{}/4/mastercard_ownership-targets.csv".format(generic_path,Year)
debitcard_targets_source = "{}/curated/vpp-mysqldb/weight-service/debit_card_ownership/{}/4/debit_card_ownership-targets.csv".format(generic_path,Year)
#discover_targets_source = ""


In [37]:
%%pyspark

#Read from ADLS and load it as a Pandas DataFrame
resps = spark.read.load(respondant_source_path, format = 'csv', sep = '|', header = True)
cards = spark.read.load(card_source_path, format = 'csv',  sep = '|', header = True)
po_codes = spark.read.load(postalCodes_source_path, format = 'csv', header = True)
cardkind  = spark.read.load(cardkind_source_path, format = 'csv',  sep = '|', header = False)
region_codes = spark.read.load(region_codes_source_path, format = 'csv', header = True)

if Legacy == "True":
    legacy_repondent = spark.read.load(legacy_repondent_source_path, format = 'csv', sep = '|', header = True)
    legacy_repondent = legacy_repondent.toPandas()

#print('Converting to Pandas.')
#Pandas Conversion

resps = resps.toPandas()
cards = cards.toPandas()
po_codes = po_codes.toPandas()
cardkind = cardkind.toPandas()
region_codes = region_codes.toPandas() 

In [64]:
resps.info()

In [65]:
legacy_repondent.info()

In [39]:
%%pyspark
#Targets Dataframes

#Read from ADLS and load it as a Pandas DataFrame
age_targets = spark.read.load(age_targets_source, format = 'csv',  header = True)
household_comp_targets = spark.read.load(household_comp_targets_source, format = 'csv', header = True)
income_targets = spark.read.load(income_targets_source, format = 'csv', header = True)
region_targets  = spark.read.load(region_targets_source, format = 'csv', header = True)
race_targets = spark.read.load(race_targets_source, format = 'csv', header = True)
amex_targets = spark.read.load(amex_targets_source, format = 'csv', header = True)
visa_targets = spark.read.load(visa_targets_source, format = 'csv', header = True)
mastercard_targets = spark.read.load(mastercard_targets_source, format = 'csv', header = True)
debitcard_targets = spark.read.load(debitcard_targets_source, format = 'csv', header = True)

#print('Converting to Pandas.')
#Pandas Conversion

age_targets = age_targets.toPandas()
household_comp_targets = household_comp_targets.toPandas()
income_targets = income_targets.toPandas()
region_targets = region_targets.toPandas()
race_targets = race_targets.toPandas()
amex_targets = amex_targets.toPandas()
visa_targets = visa_targets.toPandas()
mastercard_targets = mastercard_targets.toPandas()
debitcard_targets = debitcard_targets.toPandas()

In [56]:
###########################
# helper method definition
###########################
#Categorising values

# maps age to its predefined AgeBands
def getAgeBandID(age):
    """ This method will return the ageband(A measurement) 
        according to the responants age(Input).
        ageBands : 0 - 9
        if an ageBand == 0 its a garbage value
    """
    if(type(age) == str):
        return  0

    ageBand = 0
    if(age <= 0):
        ageBand = 0
    elif (age < 25):
        ageBand = 1
    elif (age < 30):
        ageBand = 2
    elif (age < 35):
        ageBand = 3
    elif (age < 40):
        ageBand = 4
    elif (age < 45):
        ageBand = 5
    elif (age < 50):
        ageBand = 6
    elif (age < 55):
        ageBand = 7
    elif (age < 60):
        ageBand = 8
    elif (age >=60 and age<=120):
        ageBand = 9
    
    return ageBand

#returns a list of income mapping
def getIncomeMapping(dataf):
    pun_limit = dataf.PUN_limit.astype(int).values.tolist()
    pun_limit = list(set(pun_limit))
    pun_limit.sort()

    return pun_limit    

#checks tthe inome band of respondant
def getIncomeBandID(income, mapping):
    incomeBand = 0
    if (income > mapping[-1]):
        incomeBand = len(mapping)
    else:
        for i in range(len(mapping)):
            if (income < mapping[i]):
                incomeBand = i + 1
                break
    return incomeBand


# maps race to its defined codes
def getRaceID(race):
    """This method will return the race of the respondant
        outputs = race : 1 - 4
        if the race is not in the predefined codes 
        then it will return 0"""
    raceBand = [1,2,3,4]
    
    if (race == 5):
        race = 3
    elif (race == 3 or race == 9):
        race = 4
    elif(race not in raceBand):
        race = 0
    return race

# goes through cardKind codes to get whether a certain type(Visa, MasterCard...) are owned by the respondent
def getCardOwnership(cardList, cardKindMapping):
    """ This will return 1 
            if the respondant has a card and 
            2 for not having a card"""
    
    cardowernership = 0
    for j in cardList:
        if (j in cardKindMapping):
            cardowernership = 1
        else:
            cardowernership = 2
    
    return cardowernership

#This method will return the corresponding cardmapping for the given provider 
def getCardkindMapping(cardkind_DF, providerID):
    """Returns a list of cardkinds for the given provider
       input : cardkindDataframe(containing the card provider, list of cardkinds)
               providerID - desired provider
       output : List of cardKinds"""
    cardkind_list = []
    for _, provider in cardkind_DF.iterrows(): 
        for p_code in provider['providerCode']:
            if(providerID == p_code):
                for j in provider['cardkindCode']:
                    cardkind_list.append(j)
    
    return cardkind_list


# checks paymentType attribute for Debit cards
def getDebitCardMap(cardkindDF):
    """This method will return the debit card mapping details"""
    debitCardlist = []
    
    group_paymentType = cardkindDF.groupby('paymentType')
    cardpayment_type_data = group_paymentType['paymentType','cardKindCode'].agg(lambda x: set(x))
    
    cardpay_dict = cardpayment_type_data.to_dict()
    cardpayment_type_data = pd.DataFrame(cardpay_dict)
    
    for _, i in cardpayment_type_data.iterrows():
        for paymentType in i['paymentType']:
            if(paymentType == 2):
                for cardKind in i['cardKindCode']:
                    debitCardlist.append(cardKind)

    return debitCardlist

def getHouseHoldComp(lAgmt, mSts, lwr, hsize, chCode, gender):
    #check for null values
    #lAgmt   = i['livingArrangementCode']
    #mSts    = i['maritalStatusCode']
    #lwr     = i['livingWithRelativeCode']
    #hsize   = i['householdSizeCode']
    #chCode  = i['childrenCodes']
    #gender  = i['genderCode']
    Comp = 0

    #Male/Female Only
    if(hsize == 1):
        if(mSts == 2 or mSts == 3):
            #Male only
            if(gender == 1):
                Comp = 1
            
            #Female only
            elif(gender == 2):
                Comp = 2
                
    elif(hsize == 2):
        #base logic for roommates
        if(lwr == 1 and lAgmt == 2):
            Comp = 3

        #Hus/Wife only             
        elif(mSts == 1):
            Comp = 4
            
    #Hus/Wife 
    elif(hsize > 2):
        #check the correct codes regarding the children
        #childeren < 18
        if(chCode == 5 or chCode == 3):
            Comp = 5
            
        #children 18+
        elif(chCode == 4):
            Comp = 6
            
        #base logic for roommates
        #roommates may own the place
        elif(lwr == 1 and lAgmt == 2):
            Comp = 3
            
    #If all conditions fail
    else:
        Comp = 7
    
    return Comp

#Method will drop all the colums that we don't need when there are many cols in a Dataframe
def notNeededcol(datF, colNotDrop):
    """This method will drop the unnessary columns in a dataframe
        input : dataF     - Dataframe
              : colNotDop - List of columns not to drop"""
    col = datF.columns
    coltoDrop = []
    for i in range(len(col)):
        if col[i] not in colNotDrop:
            coltoDrop.append(col[i])
    
    if coltoDrop != None:
        datF.drop(columns = coltoDrop, inplace=True) 


In [41]:
#Clean Cardkind 
cardkind = pd.DataFrame(cardkind)
cardkind.columns = ['cardKind', 'cardKindCode', 'providerCode', 'paymentType', 'quarter']
cardkind.drop(columns =['quarter'],inplace=True)
cardkind = pd.DataFrame(cardkind)  
cardkind[['cardKindCode', 'paymentType', 'providerCode']] = cardkind[['cardKindCode', 'paymentType', 'providerCode']].apply(pd.to_numeric)
cardkind.head()

In [42]:
#Transformation for cardkind Mapping
group_cardkind_Data = cardkind.groupby('providerCode')
card_kind_data = group_cardkind_Data['providerCode','cardKindCode'].agg(lambda x: set(x))
card_kind_data.columns = ['providerCode', 'cardkindCode']

In [43]:
#create cardprovider and the related cardkind maping
cardkind_dict = card_kind_data.to_dict()
card_kind_data = pd.DataFrame(cardkind_dict)
card_kind_data.head()


In [44]:
# get cardkind codes for each card criteria
VisaMapping = getCardkindMapping(card_kind_data, 4)
AmexMapping = getCardkindMapping(card_kind_data, 1)
MasterCardMapping = getCardkindMapping(card_kind_data, 3)
DiscoverMapping = getCardkindMapping(card_kind_data, 2)

incomeMapping = getIncomeMapping(income_targets)

#Since most of the payment types are null, we have to get the debitcard kind mapping through the raw data(curated data is not yet set) 
DebitCardMapping = getDebitCardMap(cardkind)

In [45]:
#regions Transformation
region_codes.columns = ['region', 'regionCode', 'gender', 'quarter', 'targets']
region_codes.drop(columns=['gender', 'quarter', 'targets'], inplace=True)
region_codes.drop_duplicates(inplace=True)

In [46]:
#Define the column that we need to keep in the dataframe
rescolnotdrop = ['id', 'age', 'genderCode', 'maritalStatusCode', 'raceCode','postalCode', 'livingArrangementCode','livingWithRelativeCode', 'householdSizeCode', 'householdIncomeCode','childrenCodes', 'empStatusCode','weight']
cardscolnotdrop = ['userId','cardKindCode','paymentTypeCode']

In [47]:
#Removing the columns we don't need
notNeededcol(resps,rescolnotdrop)
notNeededcol(cards,cardscolnotdrop)

In [48]:
#Setting column names
cards['cardKindCode'] = cards['cardKindCode'].apply(pd.to_numeric) 
cards.columns = ['id','paymentTypeCode','cardKindCode']
cards.head()

In [49]:
#Merge respondants and card details(inner join)to find thecardkind code
data = resps.merge(cards,how='inner',on="id",validate="one_to_many")
data.head(2)

In [50]:
#Clean null values and change data types
data.fillna(0, inplace=True)
data.replace('NA', 0, inplace = True)
data[['age', 'genderCode', 'maritalStatusCode', 'raceCode', 'postalCode', 'empStatusCode','livingArrangementCode','livingWithRelativeCode','householdSizeCode','householdIncomeCode', 'childrenCodes']] = data[['age','genderCode', 'maritalStatusCode', 'raceCode', 'postalCode', 'empStatusCode','livingArrangementCode','livingWithRelativeCode','householdSizeCode','householdIncomeCode', 'childrenCodes']].apply(pd.to_numeric)
#data = data.astype({'livingArrangementCode' : np.int64, 'livingWithRelativeCode':np.int64, 'paymentTypeCode': np.int64, 'childrenCodes': np.int64})
data.head(2)

In [51]:
#create a list of cardkindcodes for each respondant 
#Group by id
groupData = data.groupby('id')
#group by cardKindcode and add as a list
carddata = groupData[["cardKindCode","paymentTypeCode"]].agg(lambda x: set(x))
data = data.merge(carddata, how='inner', on="id")
data.drop(columns=['cardKindCode_x','paymentTypeCode_x'],inplace=True)

In [62]:

def getLegactHouseHoldComp(df,eid):
    comp = df[(df["userId"] == eid)]["householdComposition"].values
    print(comp)
    if len(comp)>0:
        return comp

In [63]:
getLegactHouseHoldComp(legacy_repondent,'10bcb362-b194-11eb-b12f-acde48001122')

In [52]:
# Map the respondant data according to the criteria 

def getHouseCompFromSource(eid, legacy):
    if legacy == "True":
        return getLegactHouseHoldComp(legacy_repondent,eid)
    else:
        return getHouseHoldComp(emp['livingArrangementCode'],emp['maritalStatusCode'],emp['livingWithRelativeCode'], emp['householdSizeCode'], emp['childrenCodes'],emp['genderCode'] )

newRes = []

for _, emp in data.iterrows():
    values = [
        emp['id'],
        emp['genderCode'],\
        getHouseCompFromSource(emp['id'], Legacy),\
        getAgeBandID(emp['age']),\
        getIncomeBandID(emp['householdIncomeCode'], incomeMapping),\
        emp['postalCode'],\
        getRaceID(emp['raceCode']),\
        getCardOwnership(emp['cardKindCode_y'], AmexMapping),\
        getCardOwnership(emp['cardKindCode_y'], VisaMapping),\
        getCardOwnership(emp['cardKindCode_y'], MasterCardMapping),\
        getCardOwnership(emp['cardKindCode_y'], DebitCardMapping),\
        getCardOwnership(emp['cardKindCode_y'], DiscoverMapping)
        ]
    newRes.append(values)

In [ ]:
#rename the columns
res = pd.DataFrame(newRes)
res.drop_duplicates(inplace=True)
res.columns = ['respondentId', 'genderCode','household_comp', 'age', 'income', 'postalCode', 'race', 'amex', 'visa', 'mastercard', 'debit_card', 'discover']
res.head()

In [ ]:
#Change the target region names according to the region names that are in the postal file
region_codes.replace({'Mid Atlantic', 'East North Central', 'West North Central','East South Central','West South Central'},{'Middle Atlantic','E. North Central','W. North Central','E. South Central','W. South Central'},inplace=True)
region_codes

In [ ]:
#Merged dataframe which has both postal codes and region codes
po_region = po_codes.merge(region_codes, how="inner", on='region')
po_region.drop_duplicates(inplace = True)
po_region.drop(columns = ['region','state','state_code'],inplace=True)
po_region.columns = ['postalCode','regionCode']

#append a data to map null values
po_region.append({'postalCode': 0, 'regionCode': 0},ignore_index=True)
po_region = po_region.apply(pd.to_numeric) 
po_region.head(-5)

In [ ]:
#Merge respondants data to connect the region codes  
resps = res.merge(po_region, how='left',on='postalCode')
resps.drop(columns = 'postalCode', inplace =True)
resps.head()

In [ ]:
#Replace null values with 0 and change the regionCode type to int
resps.fillna(value=0, axis = 0, inplace=True)
resps['regionCode'] = resps['regionCode'].astype(np.int64, errors='ignore') 

In [ ]:
#If any of the data has a garbage value seperate it from the original data
reslist = resps.values.tolist()
#store all the erronous values in this list
error_res = []

for i in range(len(reslist)):
   for j in range(len(reslist[i])):
      if (reslist[i][j] == 0):
         error_res.append(reslist[i])
         break

#error_respondants
error_responants = pd.DataFrame(error_res)
error_responants.columns = ['respondentId', 'gender', 'household_comp', 'age', 'income', 'race', 'amex', 'visa', 'mastercard', 'debit_card', 'discover', 'region']
error_responants.head()

In [ ]:
resps.replace(to_replace=0, value=np.nan, inplace = True)

In [ ]:
#convert any garbage values to NaN and drop it

resps.dropna(how='any',axis=0, inplace= True)
resps.columns = ['respondentId', 'gender', 'household_comp', 'age', 'income', 'race', 'amex', 'visa', 'mastercard', 'debit_card', 'discover', 'region']
resps.head(-5)

In [ ]:
resps['household_comp'] = resps['household_comp'].astype(int) 
resps['region'] = resps['region'].astype(int) 
resps['race'] = resps['race'].astype(int) 
resps.info()

In [ ]:
#this method will return the targets that needs to be achieved for a target
def retrieveDictFromDatabase(targetDF, gender_id, identifier):      
        targetDF['value'] = targetDF['value'].apply(pd.to_numeric) 
        col_not_to_drop = ['name', 'gender_id', 'band_code', 'value']
        notNeededcol(targetDF, col_not_to_drop)

        targets = {identifier: {}}
        queryDF = targetDF.query('gender_id == @gender_id')

        for _, i in queryDF.iterrows():
            try:
                i[3] = rount(i[3],2)
            except:
                pass
            targets[identifier][int(i[1])] = i[3]
        return targets

In [ ]:
#Targets dictionaries
# retrive population proportion values for each criteria from weights database
household_comp_male_targets = retrieveDictFromDatabase(household_comp_targets, "1", "household_comp")
household_comp_female_targets = retrieveDictFromDatabase(household_comp_targets, "2", "household_comp")
age_male_targets = retrieveDictFromDatabase(age_targets, "1", "age")
age_female_targets = retrieveDictFromDatabase(age_targets, "2", "age")
income_male_targets = retrieveDictFromDatabase(income_targets, "1", "income")
income_female_targets = retrieveDictFromDatabase(income_targets, "2", "income")
region_male_targets = retrieveDictFromDatabase(region_targets, "1", "region")
region_female_targets = retrieveDictFromDatabase(region_targets, "2", "region")
race_male_targets = retrieveDictFromDatabase(race_targets, "1", "race")
race_female_targets = retrieveDictFromDatabase(race_targets, "2", "race")
amex_male_targets = retrieveDictFromDatabase(amex_targets, "1", "amex")
amex_female_targets = retrieveDictFromDatabase(amex_targets, "2", "amex")
visa_male_targets = retrieveDictFromDatabase(visa_targets, "1", "visa")
visa_female_targets = retrieveDictFromDatabase(visa_targets, "2", "visa")
mastercard_male_targets = retrieveDictFromDatabase(mastercard_targets, "1", "mastercard")
mastercard_female_targets = retrieveDictFromDatabase(mastercard_targets, "2", "mastercard")
debitcard_male_targets = retrieveDictFromDatabase(debitcard_targets, "1", "debit_card")
debit_card_female_targets = retrieveDictFromDatabase(debitcard_targets, '2', "debit_card")
#discover_male_targets = retrieveDictFromDatabase("discover_ownership", "1", "discover")
#discover_female_targets = retrieveDictFromDatabase("discover_ownership", "2", "discover")

In [ ]:
len(resps)

In [ ]:
max_weight = (POPULATION/resps.shape[0]) * multiplier
print('max weight is {}'.format(max_weight))

# define weighting scheme here
scheme = qp.Rim('2019', total=POPULATION, convcrit=0.000001, cap=max_weight)

# add filters based on gender
male_filter = "gender == 1"
female_filter  = "gender == 2"

# set gender specific targets
# scheme.add_group(name='male', filter_def=male_filter, targets=male_targets)
# scheme.add_group(name='female', filter_def=female_filter, targets=female_targets)
scheme.add_group(name='male', filter_def=male_filter, targets=[
    household_comp_male_targets,
    age_male_targets,
    race_male_targets,
    income_male_targets,
    region_male_targets,
    amex_male_targets,
    visa_male_targets,
    mastercard_male_targets,
    # discover_male_targets,
    debitcard_male_targets
])
scheme.add_group(name='female', filter_def=female_filter, targets=[
    household_comp_female_targets,
    age_female_targets,
    race_female_targets,
    income_female_targets,
    region_female_targets,
    amex_female_targets,
    visa_female_targets,
    mastercard_female_targets,
    # discover_male_targets,
    debit_card_female_targets
])

# set female and male group targets
scheme.group_targets({
    'male': male_count,
    'female': female_count
})

# weighting scheme is ready!

In [ ]:
df = qp.DataSet(QUARTER + " Respondents")
df.from_components(resps)
#resp_19.to_csv("G:/weights_outputs/2019_1_res.csv")

In [ ]:
# applying the weighting
df.weight(scheme, weight_name='weight', unique_key='respondentId', inplace=True, report=True, path_report="2019 Q1", )

In [ ]:
data = df.data().copy()
data.head()

In [ ]:
data.head()

In [ ]:
# quick cehck on gender based target proportions
gender_group = data.groupby(by='gender').agg({"weight": 'sum'})
gender_group['% weight'] = gender_group.weight * 100 / gender_group.weight.sum()
gender_group

In [ ]:
# verify by race targets    
race_group = data.groupby(by=['gender', 'race']).agg({"weight": 'sum', "respondentId": 'count'})
calc_val = (race_group.weight/ race_group.weight.sum()) * 200
try:
    round(calc_val,2)
except:
    pass
race_group['% weight'] = calc_val
race_group['% unweighted'] = (race_group.respondentId/ race_group.respondentId.sum()) * 200
race_group

In [ ]:
# verify by age targets    
age_group = data.groupby(by=['gender', 'age']).agg({"weight": 'sum', "respondentId": 'count'})
calc_val_2 = (age_group.weight/ age_group.weight.sum()) * 200
try:
    round(calc_val_2,2)
except:
    pass
age_group['% weight'] = calc_val_2
age_group['% unweighted'] = (age_group.respondentId/ age_group.respondentId.sum()) * 200
age_group

In [ ]:
# verify by age targets    
age_group = data.groupby(by=['gender', 'age']).agg({"weight": 'sum', "respondentId": 'count'})
calc_val_3 = (age_group.weight/ age_group.weight.sum()) * 200
try:
    round(calc_val_3, 2)
except:
    pass
age_group['% weight'] = calc_val_3
age_group['% unweighted'] = (age_group.respondentId/ age_group.respondentId.sum()) * 200
age_group

In [ ]:
# analyse weight
w = pd.DataFrame(df.data().weight)
w.boxplot()

In [ ]:
#capping
threshold = max_weight
def capping(max_thresh, resp_weight):
    resp_weight = float(resp_weight)
    if (resp_weight > max_thresh):
        return max_thresh
    else:
        return resp_weight

In [ ]:
data['weight'] = df.data().apply(lambda row: capping(threshold,str(row['weight'])),axis =1)
data.head(-5)

In [ ]:
gender = data.groupby(by='gender').agg({'weight': 'sum'})
calc_val_4 = gender.weight * 100/gender.weight.sum()
try:
    round(calc_val_4, 0)
except:
    pass
gender['%'] = calc_val_4
gender 

In [ ]:
gender = data.groupby(by=['gender', 'race']).agg({'weight': 'sum'})
calc_val_5 =gender.weight * 200/gender.weight.sum()
try:
    round(calc_val_5, 2)
except:
    pass
gender['%'] = calc_val_5
gender 

In [ ]:
# Append two data frames and set the weighting 0 for the erronous values 
error_responants['weight'] = 0
error_responants['@1'] = 1.0
error_responants.head()

In [ ]:
error_responants['weight'] = error_responants['weight'].astype(float) 

In [ ]:
#adding two data frames
data = data.append(error_responants, ignore_index = True, sort=False)

In [ ]:
%%pyspark
data_df =spark.createDataFrame(data)

data_df = data_df.na.fill(0)

Dumping Row Data

In [ ]:
%%pyspark
#path = "{}/processed/weighted/{}/{}/weighted_respondent.csv".format(generic_path,Year,Quarter)

# Save as Table
#data_df.write.saveAsTable("Weighted_respondent_{}_{}".format(Year,Quarter))

# Dump to a csv
data_df.write.format("csv").mode("overwrite").option("header", "true").save(path)